# Екатерина - VK

In [5]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import tqdm
import time

In [3]:
url_start = 'https://ostrovok.ru/hotel/russia/baykal/?q=6307686&guests=1&page={}&price=one&sid=19ee1388-d9ac-427f-a659-169c4a056d84'

In [6]:
res = []
for p in tqdm.tqdm_notebook(range(1, 23)):
    url = url_start.format(p)
    response = requests.get(url)
    page = BeautifulSoup(response.text, 'lxml')
    hotel_name = page.find_all('h2', class_='zen-hotelcard-name')
    # baikal_distance = page.find_all('div', class_='zen-hotelcard-distance')
    #prise = page.find_all('div', class_='zen-hotelcard-rate-price-value') #при выводе доходит до 74 записей и начинает ругаться
    res.append({'page':p, 'count_hotels':len(hotel_name)})
    time.sleep(3)

<ipython-input-6-c1f012f6e982>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for p in tqdm.tqdm_notebook(range(1, 23)):


In [7]:
pd.DataFrame(res)

,page,count_hotels
0,1,25
1,2,25
2,3,25
3,4,25
4,5,25
5,6,25
6,7,25
7,8,25
8,9,25
9,10,25


Примерные точки карты, которые нужно получить. На них есть возможность перехода по другим 20 объектам

In [8]:
urls = ['https://www.google.ru/maps/search/%D0%B4%D0%BE%D1%81%D1%82%D0%BE%D0%BF%D1%80%D0%B8%D0%BC%D0%B5%D1%87%D0%B0%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D0%BE%D1%81%D1%82%D0%B8/@51.7125234,103.9733562,9z/data=!3m1!4b1!4m2!2m1!6e1',
       'https://www.google.ru/maps/search/%D0%B4%D0%BE%D1%81%D1%82%D0%BE%D0%BF%D1%80%D0%B8%D0%BC%D0%B5%D1%87%D0%B0%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D0%BE%D1%81%D1%82%D0%B8/@52.3985396,105.8933786,8z/data=!3m1!4b1!4m2!2m1!6e1',
       'https://www.google.ru/maps/search/%D0%B4%D0%BE%D1%81%D1%82%D0%BE%D0%BF%D1%80%D0%B8%D0%BC%D0%B5%D1%87%D0%B0%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D0%BE%D1%81%D1%82%D0%B8/@53.2908288,106.6973561,8z/data=!3m1!4b1!4m2!2m1!6e1',
       'https://www.google.ru/maps/search/%D0%B4%D0%BE%D1%81%D1%82%D0%BE%D0%BF%D1%80%D0%B8%D0%BC%D0%B5%D1%87%D0%B0%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D0%BE%D1%81%D1%82%D0%B8/@54.6275538,108.2470177,8z/data=!4m2!2m1!6e1',
       'https://www.google.ru/maps/search/%D0%B4%D0%BE%D1%81%D1%82%D0%BE%D0%BF%D1%80%D0%B8%D0%BC%D0%B5%D1%87%D0%B0%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D0%BE%D1%81%D1%82%D0%B8/@53.5897138,103.5288556,6.57z/data=!4m2!2m1!6e1']

In [9]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from multiprocessing.dummy import Pool
from multiprocessing import cpu_count, freeze_support
import json

In [10]:
driver = webdriver.Chrome()

In [11]:
driver.get(urls[-1])

In [14]:
# это (возможно) сами карточки достопримечательностей
driver.find_elements_by_class_name("hfpxzc")

[<selenium.webdriver.remote.webelement.WebElement (session="ae5c0f6b9f185f797c1bfbf068f2d7fa", element="9f3dbf05-e715-4172-bed7-4f2c50fa23ad")>,
 <selenium.webdriver.remote.webelement.WebElement (session="ae5c0f6b9f185f797c1bfbf068f2d7fa", element="cd425a5e-c3c3-4100-962f-bc00fb63438d")>,
 <selenium.webdriver.remote.webelement.WebElement (session="ae5c0f6b9f185f797c1bfbf068f2d7fa", element="b572435c-fb77-4d32-bb34-9761c802466b")>,
 <selenium.webdriver.remote.webelement.WebElement (session="ae5c0f6b9f185f797c1bfbf068f2d7fa", element="0c3917b7-6317-418e-b852-006491603512")>,
 <selenium.webdriver.remote.webelement.WebElement (session="ae5c0f6b9f185f797c1bfbf068f2d7fa", element="509c47ac-658c-4325-b0eb-a0c4d4d87bc7")>,
 <selenium.webdriver.remote.webelement.WebElement (session="ae5c0f6b9f185f797c1bfbf068f2d7fa", element="fc531c25-6282-45f6-a288-7163a067eee0")>]

In [22]:
# функция скрола - автоматическое прокручивание страницы до конца
def scroll(driver, path, count):
    items = []
    flag = True
    try:
        item = driver.find_element_by_xpath('//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]')
        last_height = driver.execute_script("return arguments[0].scrollHeight", item)
    except:
        flag = False


    while flag:
#         try:
#             # Scroll down to bottom
#             driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight)", item)
#             new_height = driver.execute_script("return arguments[0].scrollHeight", item)
#         except:
#             new_height = last_height
#             pass
        new_height = last_height
        # Wait to load page
        start_time = time.time()
        while new_height == last_height:
            try:
                driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight)", item)
                item = driver.find_element_by_xpath('//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]')
                # Calculate new scroll height and compare with last scroll height
                new_height = driver.execute_script("return arguments[0].scrollHeight", item)          
                items = driver.find_elements_by_xpath(path)
                if (len(items) >= count) or (time.time()-start_time)>7:
                    flag = False
                    break
            except Exception as e:
                print('3 '+e)
                flag = False
                break
        last_height = new_height

    items = driver.find_elements_by_xpath(path) 
    return items

In [21]:
# это область прокрутки
# driver.find_element_by_xpath('//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]').get_attribute('aria-label')

'Результаты по запросу "достопримечательности"'

In [27]:
# результатом скролла являются объекты - достопримечательности
result = scroll(driver, '//div[@class = "Nv2PK THOPZb CpccDe"]', 20)

In [29]:
len(result)

20

In [31]:
# Нажать на кнопку - следующая страница
driver.execute_script("arguments[0].click();", driver.find_element_by_id("ppdPk-Ej1Yeb-LgbsSe-tJiF1e"))

Тут нужно дописать - формирование таблицы из объектов - достопримечательностей - с названиями и ссылками на них, переход на другие страницы и другие ссылки из `urls`.
Затем обход сформированной таблицы с сылками на объекты для сбора большей информации. Подробнее смотреть: https://github.com/yupest/Novoznamenskiy_Krasnodar/blob/main/notebooks/2%20-%20Collecting%20metadata.ipynb 

Прим: вместо log_progress использовать библиотеку tqdm - `tqdm.tqdm_notebook` или `tqdm.notebook.tqdm`